In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install xgboost==1.5.0

In [ ]:
import xgboost as xgb
xgb.__version__

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
df_train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')

In [ ]:
df_train.columns

In [ ]:
def date_features(df):
  df['date'] = pd.to_datetime(df['date'])
  df['year'] = df.date.dt.year
  df['month'] = df.date.dt.month
  df['day'] = df.date.dt.day
  df['dayofyear'] = df.date.dt.dayofyear
  df['dayofweek'] = df.date.dt.dayofweek
  df['weekofyear'] = df.date.dt.weekofyear

  return df

In [ ]:
df_train = date_features(df_train)

In [ ]:
print(df_train.isna().sum())

In [ ]:
df_train.info()

In [ ]:
X_train = df_train.copy()

In [ ]:
print(X_train.columns)

In [ ]:
X_train.drop(columns=['id','date','sales','year','day','dayofyear','weekofyear'],axis='1',inplace=True)

In [ ]:
print(X_train.columns)

In [ ]:
X_train['store_nbr'] = X_train['store_nbr'].astype("category")
X_train['month'] = X_train['month'].astype("category")
X_train['dayofweek'] = X_train['dayofweek'].astype("category")
X_train['family'] = X_train['family'].astype("category")

In [ ]:
print(X_train.info())

In [ ]:
Y_train = df_train['sales']
Y_train = Y_train.to_frame()

In [ ]:
print(Y_train.info())

In [ ]:
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, Y_train, test_size=0.2)

In [ ]:
print(X_train_split.shape,y_train_split.shape,X_test_split.shape,y_test_split.shape)

In [ ]:
print(X_train_split.info())
print(y_train_split.info())
print(X_test_split.info())
print(y_test_split.info())

In [ ]:
import xgboost as xgb

dmat_train = xgb.DMatrix(X_train_split, label=y_train_split,enable_categorical='True')
dmat_test = xgb.DMatrix(X_test_split, label=y_test_split ,enable_categorical='True')

In [ ]:
booster = xgb.train({"tree_method": "gpu_hist",'objective': 'reg:squaredlogerror'},dmat_train,num_boost_round=10,evals=[(dmat_train, "train"), (dmat_test, "test")])

In [ ]:
df_testfile = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')

In [ ]:
df_test = date_features(df_testfile)

In [ ]:
print(df_test.isna().sum())

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
df_id = df_test['id']

In [ ]:
print(df_id)

In [ ]:
X_test = df_test.copy()

In [ ]:
X_test.drop(columns=['date','id','year','day','dayofyear','weekofyear'],axis='1',inplace=True)

In [ ]:
X_test.info()

In [ ]:
X_test['store_nbr'] = X_test['store_nbr'].astype("category")
X_test['month'] = X_test['month'].astype("category")
X_test['dayofweek'] = X_test['dayofweek'].astype("category")
X_test['family'] = X_test['family'].astype("category")

In [ ]:
X_test.info()

In [ ]:
dmat_test_predict = xgb.DMatrix(X_test, enable_categorical='True')

In [ ]:
df_result = pd.DataFrame({ "id":df_id, "sales":booster.predict(dmat_test_predict)})

In [ ]:
print(df_result.head(5))

In [ ]:
df_result.to_csv('./submission.csv',index=False)